In [1]:
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV
from sklearn.neural_network import MLPRegressor, MLPClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline


In [22]:
#CODE TO READ THE DATA

import pandas as pd

df_all_train = pd.read_json('/content/video_games_newer_all_data_with_rawg_final_for_train.json')

df_all_train.head(2)

print(df_all_train.loc[:5,'Year_of_Release' : 'reviews_count'])

   Year_of_Release  NA_Sales  EU_Sales  JP_Sales  Other_Sales  Global_Sales  \
0             2002      0.49      0.38      0.26         0.13          1.27   
1             2002      0.23      0.18      0.20         0.06          0.68   
2             2002      0.14      0.11      0.17         0.04          0.46   
3             2000      0.51      0.35      0.00         0.06          0.92   
4             2008      0.08      0.00      0.00         0.01          0.08   
5             2008      0.12      0.52      0.00         0.02          0.66   

   User_Score  added  screenshots_count  achievements_count  reddit_count  \
0         8.5     35                 18                   0             0   
1         8.9     18                 29                   0             0   
2         8.7     17                 20                   0             0   
3         6.7     97                  9                   0             0   
4         3.2      1                  0                   0  

In [23]:
df_all_train.columns

Index(['Year_of_Release', 'NA_Sales', 'EU_Sales', 'JP_Sales', 'Other_Sales',
       'Global_Sales', 'User_Score', 'added', 'screenshots_count',
       'achievements_count', 'reddit_count', 'twitch_count', 'youtube_count',
       'ratings_count', 'suggestions_count', 'additions_count',
       'game_series_count', 'reviews_count', 'Adventure', 'Puzzle',
       'Educational', 'Family', 'Platformer', 'Racing', 'Casual', 'RPG',
       'Shooter', 'Arcade', 'Fighting', 'Card', 'Massively Multiplayer',
       'Misc_Genre', 'Strategy', 'Sports', 'Board Games', 'Simulation',
       'Action', 'Indie', 'Xbox 360', 'Wii', 'Nintendo DS', 'Xbox',
       'PlayStation 3', 'Nintendo 3DS', 'Game Boy Color', 'PlayStation 5',
       'iOS', 'PlayStation 2', 'Nintendo Switch', 'Wii U', 'Xbox Series S/X',
       'Game Boy Advance', 'Nintendo 64', 'PSP', 'PlayStation', 'PC',
       'GameCube', 'Android', 'Game Boy', 'PS Vita', 'NES', 'PlayStation 4',
       'Xbox One', 'Misc_Platform'],
      dtype='object')

In [24]:
df_original_data = pd.read_csv('/content/Video_Games_newer_all_data.csv')

res = df_original_data[['Platform', 'Genre']]
res_train = df_all_train.loc[:,'Year_of_Release' : 'reviews_count']
print(type(res))
print(type(res_train))

<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>


In [25]:
df_all = pd.concat([res, res_train], axis=1)

df_all.columns

Index(['Platform', 'Genre', 'Year_of_Release', 'NA_Sales', 'EU_Sales',
       'JP_Sales', 'Other_Sales', 'Global_Sales', 'User_Score', 'added',
       'screenshots_count', 'achievements_count', 'reddit_count',
       'twitch_count', 'youtube_count', 'ratings_count', 'suggestions_count',
       'additions_count', 'game_series_count', 'reviews_count'],
      dtype='object')

In [26]:
df_all.columns

Index(['Platform', 'Genre', 'Year_of_Release', 'NA_Sales', 'EU_Sales',
       'JP_Sales', 'Other_Sales', 'Global_Sales', 'User_Score', 'added',
       'screenshots_count', 'achievements_count', 'reddit_count',
       'twitch_count', 'youtube_count', 'ratings_count', 'suggestions_count',
       'additions_count', 'game_series_count', 'reviews_count'],
      dtype='object')

In [27]:
cols = ['Platform', 'Genre']
df_all[cols] = df_all[cols].apply(lambda x: pd.factorize(x)[0] + 1)

In [28]:
#STANDARDIZE THE DATA

df_all[['Year_of_Release','added', 'screenshots_count', 
     'achievements_count',	'reddit_count',	'twitch_count',	'youtube_count',
     'ratings_count',	'suggestions_count',	'additions_count',
     'game_series_count',	'reviews_count', 'User_Score']] = StandardScaler().fit_transform(df_all[['Year_of_Release','added', 'screenshots_count', 
     'achievements_count',	'reddit_count',	'twitch_count',	'youtube_count',
     'ratings_count',	'suggestions_count',	'additions_count',
     'game_series_count',	'reviews_count', 'User_Score']])

In [29]:
print(df_all.loc[:5,'Platform' : 'reviews_count'])

   Platform  Genre  Year_of_Release  NA_Sales  EU_Sales  JP_Sales  \
0         1      1        -1.220093      0.49      0.38      0.26   
1         2      2        -1.220093      0.23      0.18      0.20   
2         1      3        -1.220093      0.14      0.11      0.17   
3         1      1        -1.683209      0.51      0.35      0.00   
4         3      4         0.169253      0.08      0.00      0.00   
5         3      5         0.169253      0.12      0.52      0.00   

   Other_Sales  Global_Sales  User_Score     added  screenshots_count  \
0         0.13          1.27    0.896010 -0.343627          -0.239689   
1         0.06          0.68    1.180726 -0.354023          -0.026918   
2         0.04          0.46    1.038368 -0.354634          -0.201003   
3         0.06          0.92   -0.385213 -0.305712          -0.413774   
4         0.01          0.08   -2.876479 -0.364419          -0.587858   
5         0.02          0.66   -0.883466 -0.361972          -0.510487   

   a

In [ ]:
#potential columns to drop

#df_all = df_all.drop(columns = ['Year_of_Release', 'screenshots_count', 'additions_count', 'game_series_count'])

In [30]:
df_all.isna().sum()

Platform                  0
Genre                     0
Year_of_Release       12917
NA_Sales              12917
EU_Sales              12917
JP_Sales              12917
Other_Sales           12917
Global_Sales          12917
User_Score            12917
added                 12917
screenshots_count     12917
achievements_count    12917
reddit_count          12917
twitch_count          12917
youtube_count         12917
ratings_count         12917
suggestions_count     12917
additions_count       12917
game_series_count     12917
reviews_count         12917
dtype: int64

In [31]:
df_all = df_all.dropna()

In [32]:
df_all.isna().sum()

Platform              0
Genre                 0
Year_of_Release       0
NA_Sales              0
EU_Sales              0
JP_Sales              0
Other_Sales           0
Global_Sales          0
User_Score            0
added                 0
screenshots_count     0
achievements_count    0
reddit_count          0
twitch_count          0
youtube_count         0
ratings_count         0
suggestions_count     0
additions_count       0
game_series_count     0
reviews_count         0
dtype: int64

In [35]:
df_classification = df_all.copy()

In [33]:
#PREPARE FOR REGRESSION

training, test = train_test_split(df_all, test_size = 0.25, random_state = 32)
features = training.drop(['NA_Sales', 'EU_Sales', 'JP_Sales', 'Other_Sales', 'Global_Sales'], axis = 1).columns

training_features, training_labels = training[features], training['NA_Sales']
test_features, test_labels = test[features], test['NA_Sales']

pipeline = Pipeline([('regressor', MLPRegressor())])
#pipeline.fit(training_features, training_labels)


In [34]:
#MLP REGRESSION WITH HALVING GRID SEARCH


estimator=pipeline


param_grid = {'regressor__hidden_layer_sizes': [(50,50,50), (50,100,50), (100,1)],
          'regressor__activation': ['relu','tanh','logistic'],
          'regressor__alpha': [0.0001, 0.05],
          'regressor__learning_rate': ['constant','adaptive'],
          'regressor__solver': ['adam']}

gsc = HalvingGridSearchCV(
    estimator,
    param_grid,
    cv=5, scoring='neg_mean_squared_error', verbose=1, n_jobs=-1)

grid_result = gsc.fit(training_features, training_labels)


best_params = grid_result.best_params_

print(best_params)

print(f'Best score: {grid_result.best_score_}')

# best_mlp = MLPRegressor(hidden_layer_sizes = best_params["hidden_layer_sizes"], 
#                         activation =best_params["activation"],
#                         solver=best_params["solver"],
#                         max_iter= 5000, n_iter_no_change = 200
#               )

# scoring = {
#            'abs_error': 'neg_mean_absolute_error',
#            'squared_error': 'neg_mean_squared_error',
#            'r2':'r2'}

# scores = cross_val_score(best_mlp, training_features, training_labels, cv=10, scoring=scoring)


n_iterations: 4
n_required_iterations: 4
n_possible_iterations: 4
min_resources_: 105
max_resources_: 2851
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 36
n_resources: 105
Fitting 5 folds for each of 36 candidates, totalling 180 fits
----------
iter: 1
n_candidates: 12
n_resources: 315
Fitting 5 folds for each of 12 candidates, totalling 60 fits
----------
iter: 2
n_candidates: 4
n_resources: 945
Fitting 5 folds for each of 4 candidates, totalling 20 fits
----------
iter: 3
n_candidates: 2
n_resources: 2835
Fitting 5 folds for each of 2 candidates, totalling 10 fits
{'regressor__activation': 'logistic', 'regressor__alpha': 0.05, 'regressor__hidden_layer_sizes': (50, 50, 50), 'regressor__learning_rate': 'constant', 'regressor__solver': 'adam'}
Best score: -1.8826184446059642


/usr/local/lib/python3.9/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


In [ ]:

import pandas as pd

df_all = pd.read_json('/content/video_games_newer_all_data_with_rawg_final_for_train.json')

df_all.head(2)

print(df_all.loc[:5,'Year_of_Release' : 'reviews_count'])

   Year_of_Release  NA_Sales  EU_Sales  JP_Sales  Other_Sales  Global_Sales  \
0             2002      0.49      0.38      0.26         0.13          1.27   
1             2002      0.23      0.18      0.20         0.06          0.68   
2             2002      0.14      0.11      0.17         0.04          0.46   
3             2000      0.51      0.35      0.00         0.06          0.92   
4             2008      0.08      0.00      0.00         0.01          0.08   
5             2008      0.12      0.52      0.00         0.02          0.66   

   User_Score  added  screenshots_count  achievements_count  reddit_count  \
0         8.5     35                 18                   0             0   
1         8.9     18                 29                   0             0   
2         8.7     17                 20                   0             0   
3         6.7     97                  9                   0             0   
4         3.2      1                  0                   0  

In [36]:
#PREPARE DATA FOR CLASSIFICATION
import numpy as np

tmp = df_classification.copy()
tmp['User_Score'] = np.where(tmp['User_Score'] < 6.0, 0, tmp['User_Score'])
tmp['User_Score'] = np.where((6.0 <= tmp['User_Score']) & ( tmp['User_Score'] < 7.0), 1 ,tmp['User_Score'])
tmp['User_Score'] = np.where((7.0 <= tmp['User_Score']) & ( tmp['User_Score'] < 7.5), 2 ,tmp['User_Score'])
tmp['User_Score'] = np.where((7.5 <= tmp['User_Score']) & ( tmp['User_Score'] < 8.0), 3 ,tmp['User_Score'])
tmp['User_Score'] = np.where((8.0 <= tmp['User_Score']) & ( tmp['User_Score'] < 8.5), 4 ,tmp['User_Score'])
tmp['User_Score'] = np.where(8.5 <= tmp['User_Score'] , 5 ,tmp['User_Score'])
tmp['User_Score'] = tmp['User_Score'].astype('category').cat.codes


In [ ]:
df_all[['Year_of_Release','added', 'screenshots_count', 
     'achievements_count',	'reddit_count',	'twitch_count',	'youtube_count',
     'ratings_count',	'suggestions_count',	'additions_count',
     'game_series_count',	'reviews_count']] = StandardScaler().fit_transform(df_all[['Year_of_Release','added', 'screenshots_count', 
     'achievements_count',	'reddit_count',	'twitch_count',	'youtube_count',
     'ratings_count',	'suggestions_count',	'additions_count',
     'game_series_count',	'reviews_count']])

In [37]:
training, test = train_test_split(tmp, test_size = 0.25, random_state = 32)
features = training.drop(['User_Score'], axis = 1).columns

training_features, training_labels = training[features], training['User_Score']
test_features, test_labels = test[features], test['User_Score']


pipeline = Pipeline([('classifier', MLPClassifier(max_iter=100))])

In [38]:
#MLP CLASSIFICATION WITH HALVING GRID SEARCH

mlp_gs = pipeline
parameter_space = {
    'classifier__hidden_layer_sizes': [(10,30,10),(20,)],
    'classifier__activation': ['tanh', 'relu'],
    'classifier__solver': ['sgd', 'adam'],
    'classifier__alpha': [0.0001, 0.05],
    'classifier__learning_rate': ['constant','adaptive'],
}


clf = HalvingGridSearchCV(mlp_gs, parameter_space, verbose=1, n_jobs=-1, cv=5, scoring='f1_macro')

grid_result = clf.fit(training_features, training_labels)


best_params = grid_result.best_params_

print(best_params)

print(f'Best score: {grid_result.best_score_}')

n_iterations: 4
n_required_iterations: 4
n_possible_iterations: 4
min_resources_: 105
max_resources_: 2851
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 32
n_resources: 105
Fitting 5 folds for each of 32 candidates, totalling 160 fits
----------
iter: 1
n_candidates: 11
n_resources: 315
Fitting 5 folds for each of 11 candidates, totalling 55 fits
----------
iter: 2
n_candidates: 4
n_resources: 945
Fitting 5 folds for each of 4 candidates, totalling 20 fits
----------
iter: 3
n_candidates: 2
n_resources: 2835
Fitting 5 folds for each of 2 candidates, totalling 10 fits
{'classifier__activation': 'relu', 'classifier__alpha': 0.05, 'classifier__hidden_layer_sizes': (20,), 'classifier__learning_rate': 'adaptive', 'classifier__solver': 'sgd'}
Best score: 1.0


/usr/local/lib/python3.9/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(
